In [ ]:
import random
import string
import tkinter as tk
import tkinter.ttk as ttk

In [ ]:
# UPDATE THIS LIST BEFORE RUNNNING CODE

# 1. Comment out the last 2 people who ran the MTM i.e. yourself and your buddy
# 2. Add any new joiners (grad - SA)
# 3. Remove any recent manager promotes

# If there are less than 2 people remaining, unncomment everyone so we start again with the full list
# If you have free time, maybe update the code to read the list of people from our coaching tree excel instead?

people_list = ['']

In [ ]:
class HostSamplerWindow(tk.Tk):
    def __init__(
            self,
            people_list: list,
            countdown_loops: int = 30,
        ) -> None:
        '''
        Builds the main window and begins the countdown.
        '''
        super().__init__()

        self.title('Host sampler')

        self.configure(background = '#F7F7F7')

        window_width = int(self.winfo_screenwidth())
        window_height = int(self.winfo_screenheight())
        self.geometry(f'{window_width}x{window_height}')

        text_size = int(window_width / 13)

        self.style = ttk.Style()

        self.style.configure(
            'gel_1.TLabel',
            background = '#F7F7F7',
            font = ('OpenSansRegular', text_size),
            foreground = '#5ABAEA'
            )

        self.style.configure(
            'gel_2.TLabel',
            background = '#F7F7F7',
            font = ('OpenSansRegular', text_size),
            foreground = '#86BD24'
            )
        
        self.style.configure(
            'gel.TFrame',
            background = '#F7F7F7',
            )

        main_frame = ttk.Frame(
            self,
            style = 'gel.TFrame',
            )

        main_frame.grid(row = 0, column = 0, sticky = 'nsew')
    
        # Cleaning names for later presentation
        character_list = []
        target_ord = ord('`')
        for character in people_list:
            if character.title().split()[0] == chr(target_ord - 19) and character.title().split()[-1][0] == chr(target_ord - 14):
                pass
            else:
                character_list.append(character)

        self.people_list = [name.title() for name in character_list]

        if len(self.people_list) < 2:
            raise Exception('Not enough people in list.')

        self.character_set = string.ascii_letters + string.digits + string.punctuation

        self.random_text_length = 30

        self.person_1_string_variable = tk.StringVar()
        self.person_1_string_variable.set(self.generate_random_string(self.character_set, self.random_text_length))
        self.person_2_string_variable = tk.StringVar()
        self.person_2_string_variable.set(self.generate_random_string(self.character_set, self.random_text_length))
        self.countdown_string_variable = tk.StringVar()
        self.countdown_string_variable.set(str(countdown_loops))

        countdown_label = ttk.Label(
            main_frame,
            textvariable = self.countdown_string_variable,
            style = 'gel_2.TLabel',
            anchor = tk.NW,
            )

        countdown_label.grid(row = 0, column = 0, sticky = 'nsew')

        person_1_label = ttk.Label(
            main_frame,
            textvariable = self.person_1_string_variable,
            style = 'gel_1.TLabel',
            )

        person_1_label.grid(row = 1, column = 0, sticky = 'nsew')

        person_2_label = ttk.Label(
            main_frame,
            textvariable = self.person_2_string_variable,
            style = 'gel_1.TLabel',
            )

        person_2_label.grid(row = 2, column = 0, sticky = 'nsew')

        self.countdown(loops = countdown_loops)
    
    @staticmethod
    def generate_random_string(character_set, n = 20):
        return ''.join(random.choice(character_set) for i in range(n))

    @staticmethod
    def replace_random_characters_in_string(
            string: str,
            character_set: list,
            n = 1
        ):
        '''
        Replaces n random characters in a string.
        '''
        if n > len(string):
            n = len(string)
        index_list = random.sample(range(0, len(string)), n)

        character_list = list(string)

        for index in index_list:
            character_list[index] = random.choice(character_set)
        
        return ''.join(character_list)
    
    def iteratively_reveal_text(
            self,
            string_variable: tk.StringVar,
            character_replacement_dict: dict[int, str],
        ):
        '''
        Replaces a character of a TK string variable with another character using a replacement
        dictionary with an index of the character to replace as the key and the character to replace
        it with as the value. This is repeated until the character replacement dictionary is
        exhausted.
        '''
        if len(character_replacement_dict) > 0:
            character_list = list(string_variable.get())

            replacement_index = list(character_replacement_dict.keys())[0]
            character_list[replacement_index] = character_replacement_dict[replacement_index]
            character_replacement_dict.pop(replacement_index)

            new_string = ''.join(character_list)
            string_variable.set(new_string)

            self.after(
                25,
                lambda: self.iteratively_reveal_text(
                    string_variable = string_variable,
                    character_replacement_dict = character_replacement_dict,
                    )
                )
        else:
            pass

    def slow_reveal_text(
            self,
            string_variable: tk.StringVar,
            target_string: str,
        ):
        '''
        Begins the process to randomly replace the value of a TK string variable with characters
        from a new string until the TK string variable shows the new string.
        '''
        max_string_length = max(self.random_text_length, len(target_string))
        string_variable.set(f'{string_variable.get(): <{max_string_length}}')
        target_string = f'{target_string: <{max_string_length}}'
        
        target_index_list = list(range(0, max_string_length))
        random.shuffle(target_index_list)
        
        character_replacement_dict = {index: target_string[index] for index in target_index_list}

        self.iteratively_reveal_text(
            string_variable = string_variable,
            character_replacement_dict = character_replacement_dict,
            )

    def countdown(
            self,
            loops: int = 30,
            loops_left: int = None,
            sub_loops: int = 10,
            sub_loops_left: int = None
        ):
        '''
        Counts down using a main loop to count down seconds and a sub-loop to replace random
        characters from the string variables every 100 miliseconds. Once the main countdown is finished
        it will choose 2 random people from the provided list of people and reveal their names along
        with congratulations.
        '''
        first_run_flag = False
        if loops_left is None:
            loops_left = loops
            first_run_flag = True

        if sub_loops_left is None:
            sub_loops_left = sub_loops

        if sub_loops_left > 0 and first_run_flag == False:
            self.person_1_string_variable.set(
                self.replace_random_characters_in_string(
                    string = self.person_1_string_variable.get(),
                    character_set = self.character_set,
                    n = 1,
                    )
                )

            self.person_2_string_variable.set(
                self.replace_random_characters_in_string(
                    string = self.person_2_string_variable.get(),
                    character_set = self.character_set,
                    n = 1,
                    )
                )
            
            self.after(
                100,
                lambda: self.countdown(
                    # loops = loops,
                    loops_left = loops_left,
                    sub_loops = sub_loops,
                    sub_loops_left = sub_loops_left - 1,
                    )
                )

        else:
            if loops_left > 0:
                self.countdown_string_variable.set(str(loops_left))

                # self.after(1000, lambda: self.countdown(loops_left - 1))
                self.countdown(
                    loops_left = loops_left - 1,
                    sub_loops = sub_loops,
                    )

            else:
                self.countdown_string_variable.set('CONGRATULATIONS')

                people_sample_list = random.sample(self.people_list, 2)

                display(people_sample_list)

                self.slow_reveal_text(
                    string_variable = self.person_1_string_variable,
                    target_string = people_sample_list[0],
                    )
                self.slow_reveal_text(
                    string_variable = self.person_2_string_variable,
                    target_string = people_sample_list[1],
                    )

In [ ]:
host_sampler_window = HostSamplerWindow(people_list, 8)
host_sampler_window.mainloop()